## 第二章 pandas基础
### Ex1：口袋妖怪数据集
1.对`HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`进行加总，验证是否为`Total`值。

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('./data/pokemon.csv')
(df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].sum(1)!=df['Total']).mean()

0.0

2. 对于`#`重复的妖怪只保留第一条记录，解决以下问题：
* 求第一属性的种类数量和前三多数量对应的种类
* 求第一属性和第二属性的组合种类
* 求尚未出现过的属性组合

In [3]:
dp_dup = df.drop_duplicates('#', keep='first')
dp_dup['Type 1'].nunique()
dp_dup['Type 1'].value_counts().index[:3]

Index(['Water', 'Normal', 'Grass'], dtype='object')

In [4]:
attr_dup = dp_dup.drop_duplicates(['Type 1', 'Type 2'])
attr_dup.shape[0]

143

In [6]:
L_full = [i+' '+j for i in df['Type 1'].unique() for j in (df['Type 1'].unique().tolist() + [''])]
L_part = [i+' '+j for i, j in zip(df['Type 1'], df['Type 2'].replace(np.nan, ''))]
res = set(L_full).difference(set(L_part))
len(res) 

188

3. 按照下述要求，构造`Series`：

* 取出物攻，超过120的替换为`high`，不足50的替换为`low`，否则设为`mid`
* 取出第一属性，分别用`replace`和`apply`替换所有字母为大写
* 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到`df`并从大到小排序

In [7]:
df['Attack'].mask(df['Attack']>120, 'high').mask(df['Attack']<50, 'low').mask((50<=df['Attack'])&(df['Attack']<=120), 'mid').head()

0    low
1    mid
2    mid
3    mid
4    mid
Name: Attack, dtype: object

In [8]:
df['Type 1'].replace({i:str.upper(i) for i in df['Type 1'].unique()})
df['Type 1'].apply(lambda x:str.upper(x)).head()

0    GRASS
1    GRASS
2    GRASS
3    GRASS
4     FIRE
Name: Type 1, dtype: object

In [9]:
df['Deviation'] = df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].apply(lambda x:np.max((x-x.median()).abs()), 1)
df.sort_values('Deviation', ascending=False).head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Deviation
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,215.0
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,207.5
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,190.0
333,306,AggronMega Aggron,Steel,NaN,630,70,140,230,60,80,50,155.0
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,145.0


### Ex2：指数加权窗口
1.

In [10]:
np.random.seed(0)
s = pd.Series(np.random.randint(-1,2,30).cumsum())
s.ewm(alpha=0.2).mean().head()
def ewm_func(x, alpha=0.2):
    win = (1-alpha)**np.arange(x.shape[0])[::-1]
    res = (win*x).sum()/win.sum()
    return res
s.expanding().apply(ewm_func).head()

0   -1.000000
1   -1.000000
2   -1.409836
3   -1.609756
4   -1.725845
dtype: float64

2.

In [12]:
s.rolling(window=4).apply(ewm_func).head()

0         NaN
1         NaN
2         NaN
3   -1.609756
4   -1.826558
dtype: float64